# 모델 설계도

#### 1. 2017 ~ 2019년의 3년간의 자료가 있는 종목은 2549 종목

#### 2. 그 중에 3년치 데이터(733 obs)가 온전하게 있는 경우는 2324 종목

#### 3. 5일 후를 예측하는 모형 생성 준비

#### 4. 그 종목들 중 10%에 대하여 Test로 분리

#### 5. 90%의 Train 데이터 중 다시 90 : 10 으로 Train과 Test를 랜덤으로 분리

#### 6. Random Forest 모형을 적용

#### 7. 10%의 Test 종목에 대하여 평균 RMSE 계산

In [1]:
# Data Loading
import pandas as pd
data = pd.read_csv("./data/df_final.csv")

# 필요한 칼럼 추출
data = data[["date", "open", "high", "low", "close", "code"]]

# Code 앞에 0 추가
data["code"] = data["code"].apply(lambda x : "{:0>6}" .format(x))

# Code type을 str로 변경
data["code"] = data["code"].apply(lambda x : str(x))

# 각 종목별 obs 개수
print("*"*22)
print("* 각 종목별 obs 개수 *")
print("*"*22)
print(data["code"].value_counts())

print("\n\n")

# 3년간의 데이터가 완전한 종목 코드
print("*"*42)
print("* 3년간 데이터가 완전한 종목 코드 733obs *")
print("*"*42)
FULL_DATA_CODE = data["code"].value_counts().index[0:2323]
print(FULL_DATA_CODE)

print("\n\n")

# 3년간의 데이터가 완전한 종목만 추출한 데이터
print("*"*48)
print("* 3년간의 데이터가 완전한 종목만 추출한 데이터 *")
print("*"*48)
data = data[data["code"].isin(FULL_DATA_CODE)]
print(data)

print("\n\n")

# RESULT DATASET 생성
FINAL_DATA = pd.DataFrame()

# 각 종목 별 Shift 생성
# 5일 전 종가로 5일 후 종가를 예측
temp_data = data                                                                                      # 임시 복사본 데이터 생성
for i in FULL_DATA_CODE[0:10] :                                                                            # 종목 순환
    data = data[data["code"] == i]                                                                   # 특정 종목의 데이터
    for s in range(1,6):                                                                             # 쉬프트 생성을 위한 반복
            data['{}{}'.format("close",s)] = data["close"].shift(s)                                        # 쉬프트 생성
    data = data.dropna().drop(["open", "high", "low", "close1", "close2", "close3", "close4"], axis=1)
    FINAL_DATA = FINAL_DATA.append(data)                                                             # 최종 데이터에 추가
    data = temp_data                                                                                 # 데이터 리셋
print("*"*49)
print("* SHIFT 생성 : 5일 전 종가로 5일 후 종가를 예측 *")
print("*"*49)
FINAL_DATA = FINAL_DATA[["code", "date", "close", "close5"]]
print("DATA Length : {}obs" .format(len(FINAL_DATA)))
print(FINAL_DATA[0:20])

C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**********************
* 각 종목별 obs 개수 *
**********************
004540    733
215000    733
019175    733
094280    733
153490    733
         ... 
570024    494
287310    494
287330    494
287300    494
269620    493
Name: code, Length: 2539, dtype: int64



******************************************
* 3년간 데이터가 완전한 종목 코드 733obs *
******************************************
Index(['004540', '215000', '019175', '094280', '153490', '140910', '009140',
       '229720', '006490', '000157',
       ...
       '214430', '122350', '053160', '145995', '115310', '043610', '060300',
       '067990', '043150', '101060'],
      dtype='object', length=2323)



************************************************
* 3년간의 데이터가 완전한 종목만 추출한 데이터 *
************************************************
               date   open   high    low  close    code
0        2017-01-02  79905  81585  79170  79800  012320
1        2017-01-03  80535  80535  79170  79800  012320
2        2017-01-04  80535  80955  76965  77175  01

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


*************************************************
* SHIFT 생성 : 5일 전 종가로 5일 후 종가를 예측 *
*************************************************
DATA Length : 7280obs
           code        date  close  close5
1040131  004540  2017-01-09   5645  5403.0
1040132  004540  2017-01-10   5626  5431.0
1040133  004540  2017-01-11   5524  5394.0
1040134  004540  2017-01-12   5543  5403.0
1040135  004540  2017-01-13   5487  5348.0
1040136  004540  2017-01-16   5459  5645.0
1040137  004540  2017-01-17   5385  5626.0
1040138  004540  2017-01-18   5506  5524.0
1040139  004540  2017-01-19   5422  5543.0
1040140  004540  2017-01-20   5459  5487.0
1040141  004540  2017-01-23   5450  5459.0
1040142  004540  2017-01-24   5236  5385.0
1040143  004540  2017-01-25   5171  5506.0
1040144  004540  2017-01-26   5217  5422.0
1040145  004540  2017-01-31   5431  5459.0
1040146  004540  2017-02-01   5431  5450.0
1040147  004540  2017-02-02   5254  5236.0
1040148  004540  2017-02-03   5431  5171.0
1040149  004540  2017-02-

In [2]:
# Data 새로 만들기
FINAL_DATA.to_csv("./data/ALLCODE_DATA.csv")

# 새로 만든 데이터 불러오기
data = pd.read_csv("./data/ALLCODE_DATA.csv")

data = data[['close', 'close5']]

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[["close5"]], data[["close"]], test_size=0.10, random_state=42)

from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)

print("="*40)
print("{:=^40}" .format(" Linear Regression "))
print("="*40)
print("Train Score : {}" .format(reg.score(X_train, y_train)))
print("Test Score : {}" .format(reg.score(X_test, y_test)))

# RMSE
error = y_test.values - reg.predict(X_test)
error_2 = error * error
import numpy as np
rmse = np.mean(error_2**0.5)
print("RMSE : {}" .format(rmse))
RRR = np.mean(y_test)
print("Percent RMSE : {}" .format(float(rmse) / RRR))
print("="*40)

========== Linear Regression ===========
Train Score : 0.9968189503705973
Test Score : 0.9949386532582914
RMSE : 552.317128129799
Percent RMSE : close    0.028568
dtype: float64
